## Watson Assistant - Extracting Workspace in an Excel Sheet
This notebook will extract all dialog nodes and intents and export them into an Excel Sheet
### Omar Megawer

In [ ]:
import pandas as pd
import numpy as np
import pixiedust
import json
import itertools
import pprint
import openpyxl

___

#### Read Watson Assistant Workspace From a Local Backup
Add the path for the downloaded Workspace here

In [ ]:
with open('Workspace.json') as json_data:
    workspaceJSON = json.load(json_data)

---

#### Read All Dialog Nodes and Extract Responses

In [ ]:
dialog_nodes = workspaceJSON['dialog_nodes']
d = []
for elem in dialog_nodes:
    conditions = elem.get('conditions')
    parent = elem.get('parent')
    nodeId = elem['dialog_node']
    title = elem.get('title')
    nodeType = elem['type']
    output = elem.get('output')
    context = elem.get('context')
    
    
    
    if output == "None":
        output = ''
        
        
    next_step = elem.get('next_step')
    row = {
        'Node_Title': title,
        'Node_Conditions': conditions,
        'Node_Type': nodeType,
        'Node_Parent': parent,
        'Next_Step': next_step,
        'Node_ID': nodeId,
        'Response': output,
        'Context': context,
        'NodeID': nodeId,
        'Node_Parent_ID': parent,
    }
    d.append(row)
df = pd.DataFrame(d, columns=['Node_Title','Node_ID', 'NodeID','Node_Conditions', 'Node_Type', 'Node_Parent', 'Node_Parent_ID','Response', 'Context'])
nodeMap = df.set_index('Node_ID')['Node_Title'].to_dict()
df['Parent_Title'] = df.Node_Parent.map(nodeMap)
df = df.drop('Node_Parent', 1)
df = df.drop('Node_ID', 1)
df = df[['Node_Title', 'NodeID','Node_Conditions','Parent_Title','Node_Parent_ID', 'Node_Type', 'Response', 'Context']]
display(df)

#### Read all Intents from Workspace

In [ ]:
questions = []
for intent in workspaceJSON['intents']:      # Response is the whole json object and it has one key value called intents with multiple intents in it
    for example in intent['examples']: # in the intents there are several intent objects containing the user examples
        row = { 
            'Question': example['text'],
            'Intent': intent['intent']
        }
        questions.append(row)
        
df2 = pd.DataFrame(questions, columns=['Question','Intent'])


---

#### Save all Dataframes to Excel Sheet

In [ ]:
writer = pd.ExcelWriter('Workspace En.xlsx')
df.to_excel(writer, "Dialog", encoding='UTF8')
df2.to_excel(writer, "Intents", encoding='UTF8')
writer.save()

---

In [ ]:
df2